# Entity Resolution for Healthcare Practice Records

## Objective
Identify whether incoming healthcare practice records already exist in the system
despite inconsistent data entry and the absence of a reliable primary key.

This notebook demonstrates:
- Data normalization
- Composite matching logic
- Fuzzy string comparison
- Match scoring and classification

In [9]:
## 1. Imports
import pandas as pd
import numpy as np
import re
from difflib import SequenceMatcher

def similarity(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [10]:
## 2. Sample Synthetic Data
#sample data, you should replace with actual data
existing = pd.DataFrame({
    "practice_name": ["Smith Medical Clinic", "Green Valley Health"],
    "practitioner_name": ["John Smith", "Aisha Bello"],
    "phone": ["(416) 555-1234", "4165559999"],
    "address": ["123 King St.", "45 Green Valley Rd"],
    "email": ["info@smithclinic.ca", "contact@gvhealth.ca"]
})

incoming = pd.DataFrame({
    "practice_name": ["SMITH medical clinic ", "Green Valley Hlth", "New Horizons Care"],
    "practitioner_name": ["Dr. John Smith", "Aisha Bello", "Michael Chen"],
    "phone": ["416-555-1234", "416 555 9999", "6475558888"],
    "address": ["123 King Street", "45 Green Valley Road", "77 Sunset Blvd"],
    "email": ["info@smithclinic.ca", "", "admin@newhorizons.ca"]
})


In [11]:
## 3. Data Normalization
#normalize functions
def normalize_text(text):
    if pd.isna(text):
        return ""
    text = text.lower().strip()
    text = re.sub(r"[^\w\s]", "", text)  # remove punctuation
    text = re.sub(r"\s+", " ", text)
    return text

def normalize_phone(phone):
    if pd.isna(phone):
        return ""
    return re.sub(r"\D", "", phone)


In [12]:
#apply normalizations
for df in [existing, incoming]:
    df["n_practice_name"] = df["practice_name"].apply(normalize_text)
    df["n_practitioner_name"] = df["practitioner_name"].apply(normalize_text)
    df["n_address"] = df["address"].apply(normalize_text)
    df["n_email"] = df["email"].apply(normalize_text)
    df["n_phone"] = df["phone"].apply(normalize_phone)


In [14]:
## 4. Match Scoring Logic
#match scoring function
def compute_match_score_with_reason(row, existing_df):
    best_score = 0
    best_reason = ""
    
    for _, ex in existing_df.iterrows():
        score = 0
        reasons = []

         # Exact matches
        if row["n_phone"] and row["n_phone"] == ex["n_phone"]:
            score += 3
            reasons.append("phone")

        if row["n_email"] and row["n_email"] == ex["n_email"]:
            score += 3
            reasons.append("email")

        # fuzzy matches using similarity
        name_sim = similarity(row["n_practice_name"], ex["n_practice_name"])
        if name_sim > 0.8:
            score += name_sim * 3
            reasons.append("practice name")

        practitioner_sim = similarity(row["n_practitioner_name"], ex["n_practitioner_name"])
        if practitioner_sim > 0.8:
            score += practitioner_sim * 2
            reasons.append("practitioner name")

        address_sim = similarity(row["n_address"], ex["n_address"])
        if address_sim > 0.8:
            score += address_sim * 2
            reasons.append("address")

        if score > best_score:
            best_score = score
            best_reason = ", ".join(reasons)

    return best_score, best_reason


In [15]:
#apply matching
incoming[["match_score", "match_reason"]] = incoming.apply(
    lambda row: pd.Series(compute_match_score_with_reason(row, existing)),
    axis=1
)


In [16]:
## 5. Record Classification
#classification logic
def classify(score):
    if score >= 7:
        return "Existing"
    elif score >= 4:
        return "Conflict – Review"
    else:
        return "New"

incoming["match_status"] = incoming["match_score"].apply(classify)


### Match Score Interpretation
Scores are weighted across multiple attributes and are not capped.
Higher scores indicate stronger confidence that records represent the same entity.


In [29]:
## 6. Output for Review
# final review output
final_output = incoming[
    ["practice_name", "practitioner_name", "phone", "address", "email",
     "match_score", "match_status"]
]

final_output


,practice_name,practitioner_name,phone,address,email,match_score,match_status
0,SMITH medical clinic,Dr. John Smith,416-555-1234,123 King Street,info@smithclinic.ca,12.431438,Existing
1,Green Valley Hlth,Aisha Bello,416 555 9999,45 Green Valley Road,,9.728070,Existing
2,New Horizons Care,Michael Chen,6475558888,77 Sunset Blvd,admin@newhorizons.ca,0.000000,New


In [18]:
#optional
final_output.to_excel("practice_match_results.xlsx", index=False)


## Limitations & Next Steps
- Thresholds may require tuning for larger datasets
- Matching logic can be extended with dates for historical tracking
- Performance optimizations may be required for very large volumes
